# Spreadsheet Analysis

In [1]:
import requests
import tempfile
import string
import re
import sys
import time
import json
import os
import csv
from urllib.parse import urlparse
from random import sample
from IPython.display import display, Markdown
from pathlib import Path
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt


In [2]:
class Table:
    def __init__(self, name, dataframe):
        self.name = name
        self.dataframe = dataframe
        self.percent_nan = self.nan_percentage()
        self.percent_bulk = 100
        self.fingerprint_flags = {
            'full_table' : False,
            'empty_top_rows' : False,
            'empty_bottom_rows': False,
            'title_row': False,
            'subtitles': False,
            'percent_bulk': None
        }
        self.fingerprint = self.fingerprint()
        self.fingerprint_analyse()
        self.empty_rows = pd.Series()
        self.empty_row_indices = []
        self.delimiter = []
       

    def check_for_empty_rows (self):
        nan_series = self.dataframe.isna().all(axis=1)
        empty_row_indices = self.dataframe.index[nan_series]
        self.empty_row_indices.append(empty_row_indices)
        if nan_series.any() == True:
            empty_rows = nan_series[nan_series == True]
            self.empty_rows = pd.Series(empty_rows)
            return True
        else:
            self.empty_rows = 0
            return False

    def nan_percentage(self):
        """ Percentage of dataframe that is NaN """
        total_cells = self.dataframe.size
        total_nan = self.dataframe.isna().sum().sum()
        return 100 * (total_nan/total_cells)
        

    def fingerprint (self):
        """ Create a fingerprint of empty and non empty rows as a list of tuples"""
        """The fingerprint is best understood as each tuple representing (n blank rows, followed by n non blank rows)"""
        #Create a series/list of value counts where count > 0 is a non empty row
        value_count = self.dataframe.notna().sum(axis=1)
        
        fingerprint = []
        blank_row_count = 0
        filled_row_count = 0
        blank_row = False

        #First check if top row is empty
        if value_count[0] == 0:
            blank_row = True
        
        for count in value_count:
            #new blank row detected, update the fingerprint list
            if count == 0 and blank_row == False:
                fingerprint.append((blank_row_count, filled_row_count))
                #reset counts
                blank_row_count = 1
                filled_row_count = 0
                blank_row = True
            #An additional (or top) blank row detected, increment the count
            elif count == 0 and blank_row == True:
                blank_row_count += 1
            #non empty row detected, update fingerprint list
            elif count != 0 and blank_row == True:
                #reset the counts
                filled_row_count = 1
                blank_row = False
            #An additional non empty row detected, increment the count
            elif count != 0 and blank_row == False:
                filled_row_count += 1
        #add last tuple
        fingerprint.append((blank_row_count, filled_row_count))
        return fingerprint

    def fingerprint_analyse(self):
        # Find bulk data
        bulk = max([item[1] for item in self.fingerprint])
        self.percent_bulk = 100 * bulk / self.dataframe.shape[0]
    
        # Store percent_bulk in fingerprint_flags
        self.fingerprint_flags['percent_bulk'] = self.percent_bulk
    
        # Check for empty top rows, regardless of fingerprint length
        if self.fingerprint[0][0] > 0:
            self.fingerprint_flags['empty_top_rows'] = True
        
        # Handle full table case
        if len(self.fingerprint) == 1 and self.fingerprint[0][0] == 0:
            self.fingerprint_flags['full_table'] = True
    
        # Split data if there are multiple sections
        if len(self.fingerprint) > 1:
            # Check if the non-nan rows are less than the bulk data
            if self.fingerprint[0][1] > 1 and self.fingerprint[0][1] < bulk:
                self.fingerprint_flags['title_row'] = True
            if self.fingerprint[-1][0] > 0 and self.fingerprint[-1][1] > 1:
                self.fingerprint_flags['empty_bottom_rows'] = True
            # Try to find potential subtitles, ignoring first tuple and last tuple
            for i in range(1, len(self.fingerprint) - 2):
                if self.fingerprint[i][1] < bulk:
                    self.fingerprint_flags['subtitles'] = True

    def get_metadata_row(self):
        """Return a dictionary of metadata for this table."""
        metadata_row = {
            'name': self.name,
            'percent_nan': self.percent_nan,
            'percent_bulk': self.fingerprint_flags.get('percent_bulk', None),
            'title_row': self.fingerprint_flags.get('title_row', False),
            'full_table': self.fingerprint_flags.get('full_table', False),
            'empty_top_rows': self.fingerprint_flags.get('empty_top_rows', False),
            'empty_rows_count': len(self.empty_row_indices),
            'empty_rows': self.empty_rows,
            'fingerprint': self.fingerprint,
            'row_count': self.dataframe.shape[0],
            'column_count': self.dataframe.shape[1]
            # Add more fields as needed
        }
        return metadata_row

    def heatmap(self):
        """ Display a heatmap of the table contents """
        # Create a mask where NaNs are 1 and non-NaNs are 0
        df = self.dataframe
        nan_mask = df.isna().astype(int)
        
        # Plotting the mask
        plt.figure(figsize=(6, 4))
        plt.imshow(nan_mask, cmap='cool', aspect='auto')  # Red for NaN (1), Green for non-NaN (0)
        plt.colorbar(label='Non-empty cells (0) / Empty cells (1)')
        plt.xticks(ticks=range(df.shape[1]), labels=df.columns)
        plt.title(f"NaN vs Non-NaN values in DataFrame {self.name}")
        plt.show()
        

In [3]:
class Tables:
    def __init__(self, directory ):
        self.tables = {}
        self.errors = []
        self.empty_rows = {
            'empty_row_count': {'has_empty_rows': 0, 'no_empty_rows': 0},
            'empty_row_files':[]
        }
    
        # Get the JSON file path from glob
        json_folder = os.path.join(directory, 'json')
        json_files = glob.glob(os.path.join(json_folder, "*_json*"))
        if json_files:
            with open(json_files[0], 'r') as f:  # Open the first file found
                self.sample_json = json.load(f)
        else:
            self.sample_json = {}

        self.load_csv_files(directory)

    #helper function (olgibbons optimise this later)
    def _locate_url(self, json, filename):
        for key, value in json.items():
            if value['file_name'] == filename:
                return key

    def load_csv_files(self, directory):
        """Load all CSV files in a directory into Table objects."""
        csv_files = glob.glob(os.path.join(directory, "*.csv*"))
        
        for file_path in csv_files:
            file_name = os.path.basename(file_path)
            delimiter = []
            try:
                """#Use sniffer to detect delimiter
                with open(file_path, newline='', encoding='latin1') as csvfile:
                    dialect = csv.Sniffer().sniff(csvfile.read(1024))
                    csvfile.seek(0)
                    d = dialect.delimiter
                    delimiter.append(d)"""
                # Load the CSV file into a dataframe and don't infer header
                df = pd.read_csv(file_path, encoding="ISO-8859-1", header=None, index_col=False, low_memory=False)
                self.tables[file_name] = Table(file_name, df)  
                #add delimiter to Table instance
                self.tables[file_name].delimiter = delimiter
                
            except Exception as e:
                # Store the error in the errors list with context
                self.errors.append({
                    'file_name': file_name,
                    'file_path': file_path,
                    'url': self._locate_url(self.sample_json, file_name),
                    'error': str(e)  # Capture the exception message
                })
                print(f"Problem reading file {file_name}: {e}")
                
    def check_empty_rows_in_all_tables(self):
        """Check if any table has empty rows."""
        self.empty_rows['empty_row_files'] = []
        has_empty_rows = 0
        no_empty_rows = 0
        
        for name, table in self.tables.items():
            has_empty = table.check_for_empty_rows()
            if has_empty:
                has_empty_rows += 1
                #add table name to list
                self.empty_rows['empty_row_files'].append(table)
            else:
                no_empty_rows += 1
    
        self.empty_rows['empty_row_count']['has_empty_rows'] = has_empty_rows
        self.empty_rows['empty_row_count']['no_empty_rows'] = no_empty_rows

    def display_empty_row_data(self):
        """Plot the summary of empty rows as a bar chart."""
        self.check_empty_rows_in_all_tables()
                
        labels = ['Tables with Empty Rows', 'Tables without Empty Rows']
        counts = [self.empty_rows['empty_row_count']['has_empty_rows'], self.empty_rows['empty_row_count']['no_empty_rows']]
        
        #data
        x = labels
        y = counts

        fig, ax = plt.subplots()

        rects = ax.bar(x, y, color = ['red', 'green'])
        ax.set_ylabel('Counts')
        ax.bar_label(rects, padding=3)
        plt.show()      

    def show_empty_tables(self):
        """View the tables with missing rows"""
        for table in self.empty_rows['empty_row_files']:
            display(Markdown(f"DataFrame: {table.name}"))
            display(table.dataframe)
            

    def display_metadata(self):
        """Plot the summary data of the downloaded files"""
        labels = self.sample_json['metadata'].keys()
        counts = self.sample_json['metadata'].values()

        fig, ax = plt.subplots()

        rects = ax.bar(labels, counts, color = ['red', 'green', 'blue', 'yellow'])
        ax.bar_label(rects, padding=3)
        plt.show()

    def show_tables_filter(self,heatmap=False, **flags):
        """
        Display the dataframes of tables where the specified fingerprint_flags match the given values.
        :param flags: Keyword arguments representing flag names and the values to check (e.g., title_row=True)
        """
        tables = []
        for table_name, table in self.tables.items():  
            # Check if all specified flags match the corresponding values in fingerprint_flags
            if all(table.fingerprint_flags.get(flag, False) == value for flag, value in flags.items()):
                print(f"Displaying table: {table.name} with flags {flags}")
                display(table.dataframe)
                tables.append(table)
            if heatmap ==True:
                table.heatmap()
        return tables

    def create_data_table(self):
        """Generate a dataframe with metadata for each table."""
        metadata = [table.get_metadata_row() for table in self.tables.values()]
        return pd.DataFrame(metadata)
        
    def get_error_log(self):
        """Return the list of errors encountered during CSV loading."""
        return self.errors

    def display_unread_tables(self):
        """ Show the raw file data for the csvs that could not be read """
        for error_file in self.errors:
            with open 

In [4]:
glob.glob('./sample_csv*')

['./sample_csvs_2024-09-27_02-15-58',
 './sample_csvs_2024-10-01_01-30-44',
 './sample_csvs_2024-10-02_21-42-02',
 './sample_csvs_2024-09-26_19-46-26',
 './sample_csvs_2024-10-11_01-20-29']

In [5]:
def combine_samples():
    samples = []
    jsons = []
    cwd = Path.cwd()
    
    # Find all directories starting with 'sample_csv*'
    sample_dirs = [d for d in cwd.glob('sample_csv*') if d.is_dir()]
    
    # Iterate through directories and collect all CSV and JSON file paths
    for dir_path in sample_dirs:
        # Collect CSV files
        csv_files = dir_path.glob('*.csv*')
        samples.extend(csv_files)  # Use extend to add CSV files to the list directly
        
        # Collect JSON files from the 'json' subdirectory
        json_folder = dir_path / 'json'  # Get the 'json' subdirectory
        if json_folder.is_dir():  # Check if the 'json' subdirectory exists
            json_files = json_folder.glob('*json')  # Collect JSON files in the 'json' folder
            jsons.extend(json_files)  # Add JSON files to the jsons list
    
    return samples, jsons

In [6]:
dir = "sample_csvs_2024-10-02_21-42-02"

In [7]:
tables1 = Tables(dir)

Problem reading file webarchive_nationalarchives_gov_uk_20180601151923_https___forestry_gov_uk_pdf_December25KFINAL_csv__FILE_December25KFINAL.csv: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2

Problem reading file webarchive_nationalarchives_gov_uk_20180601151923_https___forestry_gov_uk_pdf_june10over25k_csv__FILE_june10over25k.csv: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2

Problem reading file webarchive_nationalarchives_gov_uk___http___porthosp_nhs_uk_Downloads_Finance_2012_20Sep_20Return.csv: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2

Problem reading file manchester_gov_uk_open_downloads_file_1026_grant_payments_july_2021.csv: Error tokenizing data. C error: Expected 1 fields in line 18, saw 2



In [8]:
def locate_url(json, filename):
    for key, value in json.items():
        if value['file_name'] == filename:
            return key
        else:
            print("url not found")

In [9]:
def display_raw_csv(file_path, num_lines=10):
    """Displays the raw content of a CSV file."""
    try:
        with open(file_path, 'r', encoding='ISO-8859-1') as file:
            for _ in range(num_lines):
                print(file.readline().strip())  # Display the first `num_lines` lines
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

## Examining Fingerprints

In [10]:
fingerprints1 = [(idx, (table.fingerprint, table.dataframe.shape), table.name, table.percent_nan, table.percent_bulk, len(table.fingerprint)) for idx, table in enumerate(tables1.tables.values())]
fingerprints1

[(0,
  ([(0, 157)], (157, 424)),
  'dropbox_com_s_piqyvziovtghn8b_Contracts_20Finder_20OCDS_202016_09_14.csv',
  81.0975243360173,
  100.0,
  1),
 (1,
  ([(0, 119)], (119, 11)),
  's3_eu_west_1_amazonaws_com_datagovuk_production_ckan_organogram_organogram_nhs_blood_and_transplant_resources_2023_04_21T12_58_07Z_2023_04_21_organogram_junior.csv',
  0.0,
  100.0,
  1),
 (2,
  ([(0, 9)], (9, 20)),
  's3_eu_west_1_amazonaws_com_datagovuk_production_ckan_organogram_organogram_maritime_and_coastguard_agency_resources_2022_10_31T08_17_52Z_2022_10_31_organogram_senior.csv',
  8.333333333333332,
  100.0,
  1),
 (3,
  ([(0, 144001)], (144001, 8)),
  'admin_opendatani_gov_uk_dataset_242527e9_eeb7_4993_a63b_33e1600ddf08_resource_1d2e4318_9d29_4bd3_81f3_c1e3c8e9a1f0_download_ukhsa_coverage_report_2024_03_21.csv',
  0.0,
  100.0,
  1),
 (4,
  ([(0, 41698)], (41698, 9)),
  'data_london_gov_uk_download_modelled_estimates_of_recent_births_9698d0b1_663c_4594_8687_67469ce07e6d_actual_and_predicted_births.

In [11]:
table5 = tables1.tables['admin_opendatani_gov_uk_dataset_e37520b0_ddb4_4cfa_b53f_a9c50ef21965_resource_d16d6808_9e48_4b42_a617_e26093df34f7_download_noids_week_17_20_2024_csv.csv']

In [12]:
df = table5.dataframe

In [13]:
df

,0,1,2,3,4,5,6,7
0,Current weekly total of notifications of infec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,of the two preceding years,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Notifications of Infectious Diseases, Week 1 -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,Cumulative Totals,NaN,NaN
6,Disease,Week 20,Week 19,Week 18,Week 17,2024,2023,2022
7,NaN,NaN,NaN,NaN,NaN,Weeks,Weeks,Weeks
8,NaN,13/05 - 19/05,06/05 - 12/05,29/04 - 05/05,22/04 - 28/04,1-20,1-20,1-20
9,Acute Encephalitis/Meningitis Bacterial,0,0,0,0,13,18,6


In [14]:
pd.set_option('display.max_rows', None)

In [15]:
df = tables1.create_data_table()
new_df = df[df['empty_top_rows']==True]

In [16]:
table5.fingerprint[0][0]

0

In [17]:
table5.percent_bulk

77.35849056603773

In [18]:
table5.fingerprint_flags

{'full_table': False,
 'empty_top_rows': False,
 'empty_bottom_rows': False,
 'title_row': True,
 'subtitles': True,
 'percent_bulk': 77.35849056603773}

In [19]:
table5.fingerprint

[(0, 2), (1, 1), (1, 41), (1, 2), (1, 1), (2, 0)]

In [20]:
df = tables1.create_data_table()
df[df['percent_bulk'] <50]

,name,percent_nan,percent_bulk,title_row,full_table,empty_top_rows,empty_rows_count,empty_rows,fingerprint,row_count,column_count
18,gov_uk_government_uploads_system_uploads_attac...,70.000000,10.000000,False,False,False,0,"Series([], dtype: object)","[(0, 2), (1, 1), (1, 1), (1, 1), (1, 1), (10, 0)]",20,5
21,assets_publishing_service_gov_uk_government_up...,99.938462,0.400000,False,False,False,0,"Series([], dtype: object)","[(0, 4), (996, 0)]",1000,26
41,s3_eu_west_1_amazonaws_com_data_defra_gov_uk_A...,90.141732,17.322835,False,False,False,0,"Series([], dtype: object)","[(0, 22), (1, 1), (20, 1), (4, 1), (13, 1), (6...",127,25
47,assets_publishing_service_gov_uk_government_up...,66.666667,33.333333,False,False,True,0,"Series([], dtype: object)","[(1, 1), (1, 2), (1, 0)]",6,27
60,s3_eu_west_1_amazonaws_com_data_defra_gov_uk_A...,23.447205,31.521739,True,False,False,0,"Series([], dtype: object)","[(0, 28), (4, 1), (1, 23), (4, 1), (1, 29)]",92,7
63,assets_publishing_service_gov_uk_government_up...,99.998673,5.434783,False,False,False,0,"Series([], dtype: object)","[(0, 5), (87, 0)]",92,16384


## Creating a Filter function

We need to simplify the way we display tables according to certain criteria

In [21]:
tables1.show_tables_filter(subtitles=True)

Displaying table: admin_opendatani_gov_uk_dataset_e37520b0_ddb4_4cfa_b53f_a9c50ef21965_resource_d16d6808_9e48_4b42_a617_e26093df34f7_download_noids_week_17_20_2024_csv.csv with flags {'subtitles': True}


,0,1,2,3,4,5,6,7
0,Current weekly total of notifications of infec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,of the two preceding years,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Notifications of Infectious Diseases, Week 1 -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,Cumulative Totals,NaN,NaN
6,Disease,Week 20,Week 19,Week 18,Week 17,2024,2023,2022
7,NaN,NaN,NaN,NaN,NaN,Weeks,Weeks,Weeks
8,NaN,13/05 - 19/05,06/05 - 12/05,29/04 - 05/05,22/04 - 28/04,1-20,1-20,1-20
9,Acute Encephalitis/Meningitis Bacterial,0,0,0,0,13,18,6


Displaying table: gov_uk_government_uploads_system_uploads_attachment_data_file_670843_dft_hospitality_return_q3_2017.csv with flags {'subtitles': True}


,0,1,2,3,4
0,Minister,Date,Person or organisation that offered hospitality,Type of hospitality received,"Accompanied by spouse, family member(s) or fri..."
1,Chris Grayling,2017-09-21,Nigel Harris,National Rail Awards guest,No
2,NaN,NaN,NaN,NaN,NaN
3,John Hayes,Nil return,Nil return,Nil return,Nil return
4,NaN,NaN,NaN,NaN,NaN
5,Jesse Norman,Nil return,Nil return,Nil return,Nil return
6,NaN,NaN,NaN,NaN,NaN
7,Paul Maynard,2017-09-21,Angel Trains,Attendance at National Rail Awards reception,No
8,NaN,NaN,NaN,NaN,NaN
9,Lord Callanan,Nil return,Nil return,Nil return,Nil return


Displaying table: admin_opendatani_gov_uk_dataset_e37520b0_ddb4_4cfa_b53f_a9c50ef21965_resource_62f55697_ffb5_4655_bd91_91f5ab67d2e4_download_noids_report_2015_week_7.csv with flags {'subtitles': True}


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Current weekly total of notifications of infec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,of the two preceding years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Notifications of Infectious Diseases, Week 7, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Weekly Totals,NaN,NaN,Cumulative Totals,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
7,Disease,Week 07,Week 06,Week 05,Week 04,2015,2014,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,Weeks,Weeks,Weeks,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,01 to 07,01 to 07,01 to 07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Displaying table: s3_eu_west_1_amazonaws_com_data_defra_gov_uk_AnimalWelfare_milk_quota_purchaser_sizes_and_amounts_purchased_1994_to_2013.csv with flags {'subtitles': True}


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,Quota,< 100k,NaN,100k to 1M,NaN,1M to 3M,NaN,3M to 15M,NaN,15 to 100M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Year,No,Volume,No,Volume,No,Volume,No,Volume,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1994/95,17,0.7,23,9.1,19,35.9,18,136.8,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1995/96,17,0.7,29,14.2,9,18.5,22,162.3,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996/97,18,0.6,24,10.4,18,30.8,27,212.1,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997/98,19,0.8,28,12.9,18,28.8,25,188.5,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1998/99,22,0.7,29,12.8,20,34.1,19,129.9,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1999/00,24,0.6,24,11.1,18,28.6,21,158.0,28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2000/01,16,0.5,20,11.4,25,42.9,23,182.5,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2001/02,18,0.5,22,11.7,23,43.0,28,228.5,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Displaying table: s3_eu_west_1_amazonaws_com_data_defra_gov_uk_AnimalWelfare_cattle_movements_from_high_risk_tuberculosis_areas_may_2010.csv with flags {'subtitles': True}


,0,1,2,3,4,5,6
0,Departure Country,Departure AHDO,SECTION 1,SECTION 2,SECTION 3,SECTION 4,SECTION 5
1,ENGLAND,BURY ST EDMUNDS AHDO,7,3,346,14,0
2,NaN,CARLISLE AHDO,220,32,28,41,50
3,NaN,EXETER AHDO,545,156,0,799,110
4,NaN,GLOUCESTER AHDO,738,350,0,544,186
5,NaN,LEEDS AHDO,0,0,120,0,0
6,NaN,LEICESTER AHDO,179,50,20,268,25
7,NaN,LINCOLN AHDO,0,0,167,2,0
8,NaN,NEWCASTLE AHDO,156,4,384,5,5
9,NaN,PRESTON AHDO,0,0,196,0,0


In [22]:
len(table5.fingerprint)

6

In [23]:
table5.dataframe

,0,1,2,3,4,5,6,7
0,Current weekly total of notifications of infec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,of the two preceding years,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Notifications of Infectious Diseases, Week 1 -...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,Cumulative Totals,NaN,NaN
6,Disease,Week 20,Week 19,Week 18,Week 17,2024,2023,2022
7,NaN,NaN,NaN,NaN,NaN,Weeks,Weeks,Weeks
8,NaN,13/05 - 19/05,06/05 - 12/05,29/04 - 05/05,22/04 - 28/04,1-20,1-20,1-20
9,Acute Encephalitis/Meningitis Bacterial,0,0,0,0,13,18,6


## Testing the 1000 file sample

In [24]:
sample_dir = 'sample_csvs_2024-10-11_01-20-29'

In [25]:
big_tables = Tables(sample_dir)

Problem reading file webarchive_nationalarchives_gov_uk_20150624091829_http___ofwat_gov_uk_publications_data_gpc_prs_dat_gpc201311.csv: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2

Problem reading file webarchive_nationalarchives_gov_uk_20111129153934_http___ofwat_gov_uk_publications_data_transactions_prs_dat_transactions201408.csv: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2

Problem reading file webarchive_nationalarchives_gov_uk_20130128101641_http___data_dft_gov_uk_dft_minister_special_adviser_gifts_disclosure_special_adviser_gifts_received_dft_jan_mar2011.csv: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2

Problem reading file manchester_gov_uk_open_downloads_file_1032_grant_payments_october_2021.csv: Error tokenizing data. C error: Expected 1 fields in line 18, saw 2

Problem reading file webarchive_nationalarchives_gov_uk_20130128101641_http___data_dft_gov_uk_dft_minister_special_adviser_gifts_disclosure_special_

In [48]:
len(big_tables.errors)

28

In [41]:
big_tables.errors

[{'file_name': 'webarchive_nationalarchives_gov_uk_20150624091829_http___ofwat_gov_uk_publications_data_gpc_prs_dat_gpc201311.csv',
  'file_path': 'sample_csvs_2024-10-11_01-20-29/webarchive_nationalarchives_gov_uk_20150624091829_http___ofwat_gov_uk_publications_data_gpc_prs_dat_gpc201311.csv',
  'url': 'http://webarchive.nationalarchives.gov.uk/20150624091829/http://www.ofwat.gov.uk/publications/data/gpc/prs_dat_gpc201311.csv',
  'error': 'Error tokenizing data. C error: Expected 1 fields in line 5, saw 2\n'},
 {'file_name': 'webarchive_nationalarchives_gov_uk_20111129153934_http___ofwat_gov_uk_publications_data_transactions_prs_dat_transactions201408.csv',
  'file_path': 'sample_csvs_2024-10-11_01-20-29/webarchive_nationalarchives_gov_uk_20111129153934_http___ofwat_gov_uk_publications_data_transactions_prs_dat_transactions201408.csv',
  'url': 'http://webarchive.nationalarchives.gov.uk/20111129153934/http://www.ofwat.gov.uk/publications/data/transactions/prs_dat_transactions201408.cs

In [44]:
big_t

Spreadsheet_Analysis.ipynb       sample_csvs_2024-10-02_21-42-02/
Untitled.ipynb                   sample_csvs_2024-10-11_01-20-29/
check_urls.py                    sample_urls.py
csv_reader.ipynb                 uniq_links.txt
sample_csvs_2024-09-26_19-46-26/ uniq_links_copy.txt
sample_csvs_2024-09-27_02-15-58/ valid_urls.txt
sample_csvs_2024-10-01_01-30-44/ venv/


In [46]:
cat 'sample_csvs_2024-10-11_01-20-29/webarchive_nationalarchives_gov_uk_20111129153934_http___ofwat_gov_uk_publications_data_transactions_prs_dat_transactions201408.csv'

<!DOCTYPE html>
<html>
<head>
<style>
html, body
{
  height: 100%;
  margin: 0px;
  padding: 0px;
  border: 0px;
  overflow: hidden;
}

</style>
<script src='/static/wb_frame.js'> </script>



<!DOCTYPE html>
<html lang="en-gb" class="no-js">
<head>
    <!-- General Meta Tags -->
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta content="initial-scale = 1.0" name="viewport">
    <meta name="description" content="This Page is [ARCHIVED CONTENT] and shows what the site page http://www.ofwat.gov.uk/publications/data/transactions/prs_dat_transactions201408.csv looked like on 29 Nov 2011 at 15:39:34">

    <!-- Favicons -->
    <link rel="shortcut icon" type="image/vnd.microsoft.icon" href="/static/img/favicon.png">
    <link rel="icon" type="image/vnd.microsoft.icon" href="/static/img/favicon.png">

    <script>
      window.banner_info = {
          is_gmt: true,

          liveMsg: decodeURIComponent("Live on"),

          calendarAlt

In [47]:
big_tables.sample_json

{'https://s3-eu-west-1.amazonaws.com/datagovuk-production-ckan-organogram/organogram-department-of-health/resources/2022-04-29T09-44-15Z-2022-03-31-organogram-junior.csv': {'file_name': 's3_eu_west_1_amazonaws_com_datagovuk_production_ckan_organogram_organogram_department_of_health_resources_2022_04_29T09_44_15Z_2022_03_31_organogram_junior.csv',
  'status_code': 200,
  'is_csv': True,
  'is_csv_Al_code': True,
  'downloaded': True,
  'error': None},
 'https://www.manchester.gov.uk/open/downloads/file/1032/grant_payments_october_2021.csv': {'file_name': 'manchester_gov_uk_open_downloads_file_1032_grant_payments_october_2021.csv',
  'status_code': 200,
  'is_csv': False,
  'is_csv_Al_code': True,
  'downloaded': True,
  'error': None},
 'https://webarchive.nationalarchives.gov.uk/20180601151923/https://www.forestry.gov.uk/pdf/January2015Over25K.csv/$FILE/January2015Over25K.csv': {'file_name': 'webarchive_nationalarchives_gov_uk_20180601151923_https___forestry_gov_uk_pdf_January2015Over2